# Preprocessing
#### As a prerequisite, first run the script below before starting.

In [84]:
import json
import numpy as np
import os
import librosa
import vggish_input
import tensorflow.compat.v1 as tf_compat

tf.disable_eager_execution()

if not hasattr(tf_compat.flags, 'DEFINE_string'):
    flags = tf_compat.app.flags
else:
    flags = tf_compat.flags

if 'num_batches' not in flags.FLAGS:
    flags.DEFINE_integer('num_batches', 30, 'Number of batches of examples to feed into the model.')

if 'train_vggish' not in flags.FLAGS:
    flags.DEFINE_boolean('train_vggish', True, 'If True, allow VGGish parameters to change during training.')

if 'checkpoint' not in flags.FLAGS:
    flags.DEFINE_string('checkpoint', 'vggish_model.ckpt', 'Path to the VGGish checkpoint file.')

FLAGS = flags.FLAGS

_NUM_CLASSES = 2  # sarcastic and non sarcastic 

# 1. Test with 4 audios

### A) Data Loading and Labeling, Feature extracting

In [104]:
import json
import numpy as np
import os
import librosa
import vggish_input
import tensorflow.compat.v1 as tf_compat

tf.disable_eager_execution()

if not hasattr(tf_compat.flags, 'DEFINE_string'):
    flags = tf_compat.app.flags
else:
    flags = tf_compat.flags

if 'num_batches' not in flags.FLAGS:
    flags.DEFINE_integer('num_batches', 30, 'Number of batches of examples to feed into the model.')

if 'train_vggish' not in flags.FLAGS:
    flags.DEFINE_boolean('train_vggish', True, 'If True, allow VGGish parameters to change during training.')

if 'checkpoint' not in flags.FLAGS:
    flags.DEFINE_string('checkpoint', 'vggish_model.ckpt', 'Path to the VGGish checkpoint file.')

FLAGS = flags.FLAGS

_NUM_CLASSES = 2  # sarcastic and non sarcastic 

#  Path to JSON file and folder containing audio files
json_path = '/Users/pierrekolingba-froidevaux/Desktop/Deep_Learning/MUStARD-master/data/sarcasm_data.json'
audio_folder_path = './utterances_final_wav/'

# Load JSON Datas
with open(json_path, 'r') as f:
    labels_data = json.load(f)

# Specific list of uploaded Data
audio_files = ['2_99.wav', '2_97.wav', '2_96.wav', '2_94.wav', '2_91.wav']

def _get_examples_batch(audio_folder_path, labels_data, audio_files):
    batch_features = []
    batch_labels = []

    for file_name in audio_files:
        file_path = os.path.join(audio_folder_path, file_name)

        # Load Data and convert it in examples format
        audio, sr = librosa.load(file_path, sr=None)
        examples = vggish_input.waveform_to_examples(audio, sr)

        # Get audio file label from JSON data
        # Assuming that the file name without '.wav' is used as the key in the JSON
        is_sarcastic = labels_data[file_name.replace('.wav', '')]["sarcasm"]
        label = [1, 0] if is_sarcastic else [0, 1]

        # Add examples and label to batch
        for example in examples:
            batch_features.append(example)
            batch_labels.append(label)

    return np.array(batch_features), np.array(batch_labels)

# Use the modified function to load your 4 specific audio files
features, labels = _get_examples_batch(audio_folder_path, labels_data, audio_files)

### B) Embedding through inference

In [86]:
import vggish_slim, vggish_params, vggish_postprocess
import tensorflow.compat.v1 as tf
from sklearn.model_selection import train_test_split

# Ensure tf.compat.v1 is used for TensorFlow and disable dynamic execution
tf.disable_eager_execution()

def extract_embeddings(features):
    with tf.Graph().as_default(), tf.Session() as sess:
        # Initialize VGGish and load the checkpoint
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        # Locate input and output tensors
        features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.OUTPUT_TENSOR_NAME)

        # Initialize the post-processor
        pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)

        # Run the model to obtain embeddings
        [embedding_batch] = sess.run([embedding_tensor], feed_dict={features_tensor: features})

        # Apply post-processing - PCA (whitens the data) - Loss function way higher with it though
        postprocessed_batch = pproc.postprocess(embedding_batch)
        
        return postprocessed_batch

# Running the embedding extraction function with post-processing
embedding_batch = extract_embeddings(features)
labels = np.array(labels)

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(embedding_batch, labels, test_size=0.2, random_state=42)

INFO:tensorflow:Restoring parameters from vggish_model.ckpt


/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:318: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
I0214 13:13:01.751516 8695358720 saver.py:1413] Restoring parameters from vggish_model.ckpt


In [87]:
print(features.shape)
print(labels.shape)

(19, 96, 64)
(19, 2)


### C) Training with embedded data

In [88]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow.compat.v1 as tf
import tf_slim as slim
import vggish_params

# Ensure tf.compat.v1 is used for TensorFlow
tf.disable_eager_execution()

def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    with tf.Graph().as_default(), tf.Session() as sess:
        embeddings_input = tf.placeholder(tf.float32, shape=[None, X_train.shape[1]], name='embeddings_input')
        labels_input = tf.placeholder(tf.float32, shape=[None, _NUM_CLASSES], name='labels_input')

        # Definition of the classification model
        with tf.variable_scope('mymodel'):
            fc = slim.fully_connected(embeddings_input, 100, activation_fn=tf.nn.relu)
            logits = slim.fully_connected(fc, _NUM_CLASSES, activation_fn=None)

        # Loss and optimizer
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels_input))
        train_op = tf.train.AdamOptimizer(learning_rate=vggish_params.LEARNING_RATE, epsilon=vggish_params.ADAM_EPSILON).minimize(loss)

        # Initialization of variables
        sess.run(tf.global_variables_initializer())

        # Training loop
        for i in range(FLAGS.num_batches):
            _, loss_value = sess.run([train_op, loss], feed_dict={embeddings_input: X_train, labels_input: y_train})
            print(f'Batch {i}: Loss {loss_value}')

        # Calculation of logits on the test set
        logits_test = sess.run(logits, feed_dict={embeddings_input: X_test})

        # Conversion of logits to binary predictions
        predictions = tf.round(tf.sigmoid(logits_test))
        predicted_classes = sess.run(predictions)

        # Evaluation
        print(classification_report(y_test, predicted_classes))

# Execution of training and evaluation
train_and_evaluate_model(X_train, y_train, X_test, y_test)

Batch 0: Loss 0.000542271591257304
Batch 1: Loss 1.8461765449728773e-08
Batch 2: Loss 1.86176872529753e-11
Batch 3: Loss 8.987028379510778e-14
Batch 4: Loss 1.1381868644666916e-15
Batch 5: Loss 2.834255952190879e-17
Batch 6: Loss 1.1755497954812667e-18
Batch 7: Loss 7.295782762801428e-20
Batch 8: Loss 6.292498225113345e-21
Batch 9: Loss 7.145279936261015e-22
Batch 10: Loss 1.0254011000870219e-22
Batch 11: Loss 1.7995859423639933e-23
Batch 12: Loss 3.765552589286504e-24
Batch 13: Loss 9.189628039384557e-25
Batch 14: Loss 2.569959931714196e-25
Batch 15: Loss 8.112444636281087e-26
Batch 16: Loss 2.8527589241459094e-26
Batch 17: Loss 1.1051104658178534e-26
Batch 18: Loss 4.672428919413886e-27
Batch 19: Loss 2.136021938300662e-27
Batch 20: Loss 1.0484386097939984e-27
Batch 21: Loss 5.486469336431319e-28
Batch 22: Loss 3.043260368294678e-28
Batch 23: Loss 1.7792874717504043e-28
Batch 24: Loss 1.0916262406459211e-28
Batch 25: Loss 6.994260750963264e-29
Batch 26: Loss 4.6622578065251476e-29
Ba

/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 2. With original audio data 

### A) Data Loading, labeling, extraction

In [116]:
import json
import numpy as np
import os
import librosa
import vggish_input
import glob

# Path to the JSON file and the folder containing the audio files
json_path = '/Users/pierrekolingba-froidevaux/Desktop/Deep_Learning/MUStARD-master/data/sarcasm_data.json'
audio_folder_path = './utterances_final_wav/'

# Load the JSON data
with open(json_path, 'r') as f:
    labels_data = json.load(f)

# Function to generate batches of examples and labels
def get_all_examples(audio_folder_path, labels_data):
    audio_files = glob.glob(os.path.join(audio_folder_path, '*.wav'))
    all_features = []
    all_labels = []

    for file_path in audio_files:
        file_name = os.path.basename(file_path).replace('.wav', '')

        # Load the audio and convert it into an example
        audio, sr = librosa.load(file_path, sr=None)
        examples = vggish_input.waveform_to_examples(audio, sr)

        # Get the audio file's label from the JSON data
        is_sarcastic = labels_data[file_name]["sarcasm"]
        label = [1, 0] if is_sarcastic else [0, 1]  # [sarcasm, non-sarcasm]

        for example in examples:
            all_features.append(example)
            all_labels.append(label)

    return np.array(all_features), np.array(all_labels)

features, labels = get_all_examples(audio_folder_path, labels_data)

### B) Embedding through inference

In [117]:
import vggish_slim, vggish_params, vggish_postprocess
import tensorflow.compat.v1 as tf
from sklearn.model_selection import train_test_split

# Ensure tf.compat.v1 is used for TensorFlow and disable dynamic execution
tf.disable_eager_execution()

def extract_embeddings(features):
    with tf.Graph().as_default(), tf.Session() as sess:
        # Initialize VGGish and load the checkpoint
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        # Locate input and output tensors
        features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.OUTPUT_TENSOR_NAME)

        # Initialize the post-processor
        pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)

        # Run the model to obtain embeddings
        [embedding_batch] = sess.run([embedding_tensor], feed_dict={features_tensor: features})

        # Apply post-processing - PCA (whitens the data) - Loss function way higher with it though
        postprocessed_batch = pproc.postprocess(embedding_batch)
        
        return postprocessed_batch

# Running the embedding extraction function with post-processing
embedding_batch = extract_embeddings(features)
labels = np.array(labels)

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(embedding_batch, labels, test_size=0.2, random_state=42, stratify=labels)

INFO:tensorflow:Restoring parameters from vggish_model.ckpt


/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:318: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
I0214 14:21:43.427644 8695358720 saver.py:1413] Restoring parameters from vggish_model.ckpt


In [107]:
print(embedding_batch.shape)
print(labels)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

(3475, 128)
[[0 1]
 [0 1]
 [0 1]
 ...
 [1 0]
 [1 0]
 [1 0]]
[[ 41 124 227 ... 117 255   0]
 [ 57 234 255 ... 231 185 255]
 [114 127 229 ... 255 255  99]
 ...
 [ 61 147 250 ...   6 255 255]
 [ 83 135 226 ...   0 255   0]
 [ 93  84 220 ...  74 112 213]]
[[ 79  78 201 ... 228  69 154]
 [  0 255 255 ...   0 255 255]
 [ 96 133 255 ...   0 255 255]
 ...
 [ 53 149 230 ...   0 255 197]
 [139  76 197 ... 231   0 255]
 [107  84 201 ... 193  75 113]]
[[0 1]
 [0 1]
 [1 0]
 ...
 [0 1]
 [0 1]
 [1 0]]
[[1 0]
 [0 1]
 [1 0]
 ...
 [1 0]
 [0 1]
 [1 0]]


### C) Training with embedded data

In [118]:
import tensorflow.compat.v1 as tf
import tf_slim as slim
import vggish_params

# Ensure tf.compat.v1 is used for TensorFlow
tf.disable_eager_execution()

def train_and_evaluate_model(X_train, y_train, X_test, y_test, _NUM_CLASSES):
    with tf.Graph().as_default(), tf.Session() as sess:
        # Placeholders for embeddings and labels
        embeddings_input = tf.placeholder(tf.float32, shape=[None, X_train.shape[1]], name='embeddings_input')
        labels_input = tf.placeholder(tf.float32, shape=[None, _NUM_CLASSES], name='labels_input')

        # Definition of the new top layer for VGGish
        with tf.variable_scope('vggish_adapted'):
            # Adding a layer of Global Average Pooling - Dimensions are different, Gao's paper not specific about it
            #net = tf.reduce_mean(embeddings_input, axis=[1, 2], keepdims=True, name='global_avg_pool')
            net = embeddings_input  # Correction here
            
            # First fully connected layer adapted
            net = slim.fully_connected(net, 4096, activation_fn=tf.nn.relu, scope='fc_adapted1')
            
            # Second fully connected layer adapted
            net = slim.fully_connected(net, 4096, activation_fn=tf.nn.relu, scope='fc_adapted2')
            
            # Logits layer for classification adapted to the new task
            logits = slim.fully_connected(net, _NUM_CLASSES, activation_fn=None, scope='logits')

        # Loss and optimizer
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels_input))
        train_op = tf.train.AdamOptimizer(learning_rate=vggish_params.LEARNING_RATE, epsilon=vggish_params.ADAM_EPSILON).minimize(loss)

        # Initialization of variables
        sess.run(tf.global_variables_initializer())

        # Training loop
        for i in range(FLAGS.num_batches):
            _, loss_value = sess.run([train_op, loss], feed_dict={embeddings_input: X_train, labels_input: y_train})
            print(f'Batch {i}: Loss {loss_value}')

        # Calculation of logits on the test set
        logits_test = sess.run(logits, feed_dict={embeddings_input: X_test})

        # Conversion of logits to binary predictions
        predictions = tf.round(tf.sigmoid(logits_test))
        predicted_classes = sess.run(predictions)

        # Evaluation
        print(classification_report(y_test, predicted_classes))

train_and_evaluate_model(X_train, y_train, X_test, y_test, _NUM_CLASSES)

/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Batch 0: Loss 17.407608032226562
Batch 1: Loss 66.51174926757812
Batch 2: Loss 62.00385665893555
Batch 3: Loss 57.76488494873047
Batch 4: Loss 29.130760192871094
Batch 5: Loss 9.929426193237305
Batch 6: Loss 30.343368530273438
Batch 7: Loss 28.99596405029297
Batch 8: Loss 26.74911117553711
Batch 9: Loss 14.744040489196777
Batch 10: Loss 8.207809448242188
Batch 11: Loss 16.846784591674805
Batch 12: Loss 20.67616844177246
Batch 13: Loss 17.20814323425293
Batch 14: Loss 9.673805236816406
Batch 15: Loss 7.701658725738525
Batch 16: Loss 9.110270500183105
Batch 17: Loss 11.665075302124023
Batch 18: Loss 12.829596519470215
Batch 19: Loss 7.583329677581787
Batch 20: Loss 5.015645980834961
Batch 21: Loss 6.973839282989502
Batch 22: Loss 7.815910339355469
Batch 23: Loss 8.395711898803711
Batch 24: Loss 5.433155536651611
Batch 25: Loss 2.6407253742218018
Batch 26: Loss 6.027275085449219
Batch 27: Loss 6.338827610015869
Batch 28: Loss 3.2198662757873535
Batch 29: Loss 3.1518027782440186
          

In [115]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("Num batches:", FLAGS.num_batches)
print(classification_report(y_test, predicted_classes))

X_train shape: (2769, 128)
y_train shape: (2769, 2)
X_test shape: (693, 128)
y_test shape: (693, 2)
Num batches: 30


NameError: name 'predicted_classes' is not defined

# 3. With denoised audio data

### A) Data Loading, labeling, extraction

In [119]:
import json
import numpy as np
import os
import librosa
import vggish_input
import glob

# Path to the JSON file and the folder containing the audio files
json_path = '/Users/pierrekolingba-froidevaux/Desktop/Deep_Learning/MUStARD-master/data/sarcasm_data.json'
audio_folder_path = './raw_audio_pcm_f32le_16kHz_denoised/'

# Load the JSON data
with open(json_path, 'r') as f:
    labels_data = json.load(f)

# Function to generate batches of examples and labels
def get_all_examples(audio_folder_path, labels_data):
    audio_files = glob.glob(os.path.join(audio_folder_path, '*.wav'))
    all_features = []
    all_labels = []

    for file_path in audio_files:
        file_name = os.path.basename(file_path).replace('.wav', '')

        # Load the audio and convert it into an example
        audio, sr = librosa.load(file_path, sr=None)
        examples = vggish_input.waveform_to_examples(audio, sr)

        # Get the audio file's label from the JSON data
        is_sarcastic = labels_data[file_name]["sarcasm"]
        label = [1, 0] if is_sarcastic else [0, 1]  # [sarcasm, non-sarcasm]

        for example in examples:
            all_features.append(example)
            all_labels.append(label)

    return np.array(all_features), np.array(all_labels)

features, labels = get_all_examples(audio_folder_path, labels_data)

### B) Embedding through inference

In [120]:
import vggish_slim, vggish_params, vggish_postprocess
import tensorflow.compat.v1 as tf
from sklearn.model_selection import train_test_split

# Ensure tf.compat.v1 is used for TensorFlow and disable dynamic execution
tf.disable_eager_execution()

def extract_embeddings(features):
    with tf.Graph().as_default(), tf.Session() as sess:
        # Initialize VGGish and load the checkpoint
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        # Locate input and output tensors
        features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.OUTPUT_TENSOR_NAME)

        # Initialize the post-processor
        pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)

        # Run the model to obtain embeddings
        [embedding_batch] = sess.run([embedding_tensor], feed_dict={features_tensor: features})

        # Apply post-processing - PCA (whitens the data) - Loss function way higher with it though
        postprocessed_batch = pproc.postprocess(embedding_batch)
        
        return postprocessed_batch

# Running the embedding extraction function with post-processing
embedding_batch = extract_embeddings(features)
labels = np.array(labels)

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(embedding_batch, labels, test_size=0.2, random_state=42, stratify=labels)

INFO:tensorflow:Restoring parameters from vggish_model.ckpt


/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:318: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
I0214 14:33:42.046692 8695358720 saver.py:1413] Restoring parameters from vggish_model.ckpt


In [113]:
print(embedding_batch.shape)
print(labels)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

(3462, 128)
[[0 1]
 [0 1]
 [0 1]
 ...
 [1 0]
 [1 0]
 [1 0]]
[[ 67  56 120 ... 159 124 255]
 [ 93 101  88 ...  82 127 205]
 [ 85  60 165 ...   0 101 255]
 ...
 [ 35  76 115 ...  56 255   0]
 [ 56 120  96 ... 255 255  63]
 [132  88 143 ... 139  86 255]]
[[ 78  84 127 ... 247   0 140]
 [101  42 175 ...  11 255  69]
 [ 41  80 122 ... 208 129 153]
 ...
 [ 70  88 151 ...   0 127 240]
 [ 90  81  76 ...   0 189 255]
 [ 74 106 117 ...  69   0 255]]
[[0 1]
 [0 1]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]
[[0 1]
 [1 0]
 [1 0]
 ...
 [0 1]
 [1 0]
 [0 1]]


### C) Training with embedded data

In [121]:
import tensorflow.compat.v1 as tf
import tf_slim as slim
import vggish_params

# Ensure tf.compat.v1 is used for TensorFlow
tf.disable_eager_execution()

def train_and_evaluate_model(X_train, y_train, X_test, y_test, _NUM_CLASSES):
    with tf.Graph().as_default(), tf.Session() as sess:
        # Placeholders for embeddings and labels
        embeddings_input = tf.placeholder(tf.float32, shape=[None, X_train.shape[1]], name='embeddings_input')
        labels_input = tf.placeholder(tf.float32, shape=[None, _NUM_CLASSES], name='labels_input')

        # Definition of the new top layer for VGGish
        with tf.variable_scope('vggish_adapted'):
            # Adding a layer of Global Average Pooling - Dimensions are different, Gao's paper not specific about it
            #net = tf.reduce_mean(embeddings_input, axis=[1, 2], keepdims=True, name='global_avg_pool')
            net = embeddings_input  # Correction here
            
            # First adapted fully connected layer
            net = slim.fully_connected(net, 4096, activation_fn=tf.nn.relu, scope='fc_adapted1')
            
            # Second adapted fully connected layer
            net = slim.fully_connected(net, 4096, activation_fn=tf.nn.relu, scope='fc_adapted2')
            
            # Logits layer for classification adapted to the new task
            logits = slim.fully_connected(net, _NUM_CLASSES, activation_fn=None, scope='logits')

        # Loss and optimizer
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels_input))
        train_op = tf.train.AdamOptimizer(learning_rate=vggish_params.LEARNING_RATE, epsilon=vggish_params.ADAM_EPSILON).minimize(loss)

        # Initialization of variables
        sess.run(tf.global_variables_initializer())

        # Training loop
        for i in range(FLAGS.num_batches):
            _, loss_value = sess.run([train_op, loss], feed_dict={embeddings_input: X_train, labels_input: y_train})
            print(f'Batch {i}: Loss {loss_value}')

        # Calculation of logits on the test set
        logits_test = sess.run(logits, feed_dict={embeddings_input: X_test})

        # Conversion of logits to binary predictions
        predictions = tf.round(tf.sigmoid(logits_test))
        predicted_classes = sess.run(predictions)

        # Evaluation
        print(classification_report(y_test, predicted_classes))

train_and_evaluate_model(X_train, y_train, X_test, y_test, _NUM_CLASSES)

/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1697: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Batch 0: Loss 7.961370468139648
Batch 1: Loss 66.02604675292969
Batch 2: Loss 69.88257598876953
Batch 3: Loss 48.334922790527344
Batch 4: Loss 27.800418853759766
Batch 5: Loss 5.777182102203369
Batch 6: Loss 29.18089485168457
Batch 7: Loss 41.489315032958984
Batch 8: Loss 39.91816711425781
Batch 9: Loss 31.94264030456543
Batch 10: Loss 17.72698211669922
Batch 11: Loss 2.9278132915496826
Batch 12: Loss 13.160613059997559
Batch 13: Loss 19.01643180847168
Batch 14: Loss 21.5030574798584
Batch 15: Loss 20.49531364440918
Batch 16: Loss 16.61652183532715
Batch 17: Loss 11.16903305053711
Batch 18: Loss 3.711963415145874
Batch 19: Loss 6.547635555267334
Batch 20: Loss 12.289713859558105
Batch 21: Loss 13.327323913574219
Batch 22: Loss 10.96654987335205
Batch 23: Loss 5.512005805969238
Batch 24: Loss 2.597316265106201
Batch 25: Loss 6.385324954986572
Batch 26: Loss 8.423666954040527
Batch 27: Loss 8.374922752380371
Batch 28: Loss 6.657341957092285
Batch 29: Loss 3.4592630863189697
             

/Users/pierrekolingba-froidevaux/anaconda3/envs/new_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
